현재 튜토리얼을 작성하는 버전은 soynlp=0.0.493 입니다. 

In [1]:
import sys
sys.path.append('../')
import soynlp

print(soynlp.__version__)

0.0.493


tagger 에서 이용하는 단어의 품사는 다음과 같습니다. 

In [2]:
from soynlp.postagger.tagset import tagset
from pprint import pprint
pprint(tagset)

{'Adjective': '형용사',
 'Adverb': '부사',
 'Determiner': '관형사',
 'Exclamation': '감탄사',
 'Josa': '조사',
 'Noun': '명사',
 'Numeral': '수사',
 'Pronoun': '대명사',
 'Symbol': '기호',
 'Verb': '동사'}


SimpleTagger 는 사전 기반으로 작동하는 품사 판별기 입니다. 이는 세 가지 구성 요소로 이뤄져 있습니다. (1) 주어진 문장에서 단어열 후보를 생성하는 generator, (2) 여러 개의 단어열 후보 중에서 가장 적절한 후보를 선택하는 evaluator, (3) 처리하지 못한 단어들에 대한 후처리 기능을 담당할 postprocessor. 그리고 (1) 의 generator 는 사전 기반으로 작동하기 때문에 dictionary 가 필요합니다. 아래는 dict[str]=set 형식의 input 을 이용하는 Dictionary 를 만드는 과정입니다. 

    from soynlp.pos import Dictionary
    dictionary = Dictionary(YOUR_DICT)

In [3]:
from soynlp.postagger import Dictionary
from soynlp.postagger import LRTemplateMatcher
from soynlp.postagger import LREvaluator
from soynlp.postagger import SimpleTagger
from soynlp.postagger import UnknowLRPostprocessor

pos_dict = {
    'Adverb': {'너무', '매우'}, 
    'Noun': {'너무너무너무', '아이오아이', '아이', '노래', '오', '이', '고양'},
    'Josa': {'는', '의', '이다', '입니다', '이', '이는', '를', '라', '라는'},
    'Verb': {'하는', '하다', '하고'},
    'Adjective': {'예쁜', '예쁘다'},
    'Exclamation': {'우와'}    
}

dictionary = Dictionary(pos_dict)

사용자에 의하여 입력된 pos\_dict 는 dictionary.pos\_dict 에 저장되어 있습니다. 

In [4]:
dictionary.pos_dict

{'Adverb': {'너무', '매우'},
 'Noun': {'고양', '너무너무너무', '노래', '아이', '아이오아이', '오', '이'},
 'Josa': {'는', '라', '라는', '를', '의', '이', '이는', '이다', '입니다'},
 'Verb': {'하고', '하는', '하다'},
 'Adjective': {'예쁘다', '예쁜'},
 'Exclamation': {'우와'}}

get_pos(word) 함수는 주어진 단어 word 에 대하여 등록되어 있는 모든 품사를 list 형식으로 return 합니다.

In [5]:
print(dictionary.get_pos('아이오아이'))
print(dictionary.get_pos('이'))

['Noun']
['Noun', 'Josa']


word_is_tag(word, tag) 함수는 주어진 단어 word 가 품사 tag 인지 확인하는 함수입니다. 

In [6]:
print(dictionary.word_is_tag('아이오아이', 'Noun'))
print(dictionary.word_is_tag('아이오아이', '명사'))

True
False


add_words(tag, words) 는 사전에 품사가 tag 인 단어들을 추가하는 기능입니다. words 는 하나의 str 이어도 되며, 여러 개의 단어로 이뤄진 collection of str 입니다. 

In [12]:
print('# Add a word with a tag')
dictionary.add_words('Noun', '앙순이')
pprint(dictionary.pos_dict)

print('\n# Add words with a tag')
dictionary.add_words('Noun', ['워너원', '아이돌'])
pprint(dictionary.pos_dict)

# Add a word with a tag
{'Adjective': {'예쁘다', '예쁜'},
 'Adverb': {'매우', '너무'},
 'Exclamation': {'우와'},
 'Josa': {'이', '라는', '는', '이다', '를', '라', '입니다', '의', '이는'},
 'Noun': {'이', '아이', '노래', '고양', '아이오아이', '너무너무너무', '앙순이', '오'},
 'Verb': {'하고', '하는', '하다'}}

# Add words with a tag
{'Adjective': {'예쁘다', '예쁜'},
 'Adverb': {'매우', '너무'},
 'Exclamation': {'우와'},
 'Josa': {'이', '라는', '는', '이다', '를', '라', '입니다', '의', '이는'},
 'Noun': {'이', '앙순이', '아이', '노래', '아이오아이', '워너원', '고양', '아이돌', '너무너무너무', '오'},
 'Verb': {'하고', '하는', '하다'}}


현재 사전에 등록되지 않은 품사는 입력되지 않습니다. 품사 태그로 오탈자가 입력되는 것을 방지하기 위함입니다. 

In [13]:
dictionary.add_words('Name', 'lovit')

ValueError: Check your tag or use add_words(tag, words, force=True)

하지만 force=True 로 설정하면 현재 등록되지 않은 품사라 하여도 사전에 추가할 수 있습니다. 

In [14]:
dictionary.add_words('Name', 'lovit', force=True)
pprint(dictionary.pos_dict)

{'Adjective': {'예쁘다', '예쁜'},
 'Adverb': {'매우', '너무'},
 'Exclamation': {'우와'},
 'Josa': {'이', '라는', '는', '이다', '를', '라', '입니다', '의', '이는'},
 'Name': {'lovit'},
 'Noun': {'이', '앙순이', '아이', '노래', '아이오아이', '워너원', '고양', '아이돌', '너무너무너무', '오'},
 'Verb': {'하고', '하는', '하다'}}


remove_words(tag, words) 는 품사 tag 에 해당하는 words 를 사전에서 제거합니다. 

In [15]:
dictionary.remove_words('Noun', {'앙순이', '워너원'} )
pprint(dictionary.pos_dict)

{'Adjective': {'예쁘다', '예쁜'},
 'Adverb': {'매우', '너무'},
 'Exclamation': {'우와'},
 'Josa': {'이', '라는', '는', '이다', '를', '라', '입니다', '의', '이는'},
 'Name': {'lovit'},
 'Noun': {'이', '아이', '노래', '아이오아이', '고양', '아이돌', '너무너무너무', '오'},
 'Verb': {'하고', '하는', '하다'}}


remove_words(tag) 만 입력할 경우, 해당 품사를 모두 제거합니다. 

In [16]:
dictionary.remove_words('Noun')
pprint(dictionary.pos_dict)

{'Adjective': {'예쁘다', '예쁜'},
 'Adverb': {'매우', '너무'},
 'Exclamation': {'우와'},
 'Josa': {'이', '라는', '는', '이다', '를', '라', '입니다', '의', '이는'},
 'Name': {'lovit'},
 'Verb': {'하고', '하는', '하다'}}


Dictionary 를 이용하여 LRTemplateMatcher 를 만듭니다. LRTemplateMatcher 는 단어열 후보를 생성합니다. 

In [17]:
sent = '너무너무너무는아이오아이의노래입니다!!'

pos_dict = {
    'Adverb': {'너무', '매우'}, 
    'Noun': {'너무너무너무', '아이오아이', '아이', '노래', '오', '이', '고양'},
    'Josa': {'는', '의', '이다', '입니다', '이', '이는', '를', '라', '라는'},
    'Verb': {'하는', '하다', '하고'},
    'Adjective': {'예쁜', '예쁘다'},
    'Exclamation': {'우와'}    
}

dictionary = Dictionary(pos_dict)
generator = LRTemplateMatcher(dictionary)
pprint(generator.generate(sent))

[LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=0, m=2, e=2),
 LR(l='너무너무너무', l_tag='Noun', r='', r_tag=None, b=0, m=6, e=6),
 LR(l='너무너무너무', l_tag='Noun', r='는', r_tag='Josa', b=0, m=6, e=7),
 LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=2, m=4, e=4),
 LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=4, m=6, e=6),
 LR(l='아이', l_tag='Noun', r='', r_tag=None, b=7, m=9, e=9),
 LR(l='아이오아이', l_tag='Noun', r='', r_tag=None, b=7, m=12, e=12),
 LR(l='아이오아이', l_tag='Noun', r='의', r_tag='Josa', b=7, m=12, e=13),
 LR(l='아이', l_tag='Noun', r='', r_tag=None, b=10, m=12, e=12),
 LR(l='아이', l_tag='Noun', r='의', r_tag='Josa', b=10, m=12, e=13),
 LR(l='노래', l_tag='Noun', r='', r_tag=None, b=13, m=15, e=15),
 LR(l='노래', l_tag='Noun', r='입니다', r_tag='Josa', b=13, m=15, e=18)]


generator.generate(sentence) 의 결과는 LR 이라는 namedtuple 입니다. 한국어의 어절을 명사/형용사/동사/부사/감탄사 의 L parts (left-subsection) 와 조사, 동사, 형용사 (동사와 형용사는 정확히는 전성어미, 이론 튜토리얼 참조) 로 이뤄져있습니다. Template 에 있는 패턴과 사전을 이용하여 매칭이 되는 후보들을 생성합니다. 

b, m, e 는 문장 내에서 L 의 위치 (b, m) 와 R 의 위치 (m, e) 입니다. 

In [18]:
generator.generate(sent)[0]

LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=0, m=2, e=2)

In [20]:
generator.templates

{'Noun': ('Josa', 'Verb', 'Adjective')}

Templates 와 Evaluator 를 이용하여 Tagger 를 만듭니다. 

In [21]:
evaluator = LREvaluator()
postprocessor = UnknowLRPostprocessor()

tagger = SimpleTagger(generator, evaluator, postprocessor)
tagger.tag(sent)

[('너무너무너무', 'Noun'),
 ('는', 'Josa'),
 ('아이오아이', 'Noun'),
 ('의', 'Josa'),
 ('노래', 'Noun'),
 ('입니다', 'Josa'),
 ('!!', None)]

postprocessor 가 입력되지 않으면, 사전 매칭이 되지 않은 단어들은 출력되지 않습니다. 

In [23]:
SimpleTagger(generator, evaluator).tag(sent)

[('너무너무너무', 'Noun'),
 ('는', 'Josa'),
 ('아이오아이', 'Noun'),
 ('의', 'Josa'),
 ('노래', 'Noun'),
 ('입니다', 'Josa')]

debug mode 로 tag() 를 실행할 경우 문장 내의 단어열 뿐 아니라 디버깅용 LR 후보 리스트들이 출력됩니다. 

In [24]:
tags, debugs = tagger.tag(sent, debug=True)

In [25]:
pprint(tags)

[('너무너무너무', 'Noun'),
 ('는', 'Josa'),
 ('아이오아이', 'Noun'),
 ('의', 'Josa'),
 ('노래', 'Noun'),
 ('입니다', 'Josa'),
 ('!!', None)]


In [26]:
pprint(debugs)

[[(LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=0, m=2, e=2), 0.4),
  (LR(l='너무너무너무', l_tag='Noun', r='', r_tag=None, b=0, m=6, e=6),
   0.9000000000000001),
  (LR(l='너무너무너무', l_tag='Noun', r='는', r_tag='Josa', b=0, m=6, e=7),
   1.1500000000000001),
  (LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=2, m=4, e=4), 0.4),
  (LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=4, m=6, e=6), 0.4),
  (LR(l='아이', l_tag='Noun', r='', r_tag=None, b=7, m=9, e=9), 0.5),
  (LR(l='아이오아이', l_tag='Noun', r='', r_tag=None, b=7, m=12, e=12), 0.8),
  (LR(l='아이오아이', l_tag='Noun', r='의', r_tag='Josa', b=7, m=12, e=13), 1.05),
  (LR(l='아이', l_tag='Noun', r='', r_tag=None, b=10, m=12, e=12), 0.5),
  (LR(l='아이', l_tag='Noun', r='의', r_tag='Josa', b=10, m=12, e=13),
   0.7499999999999999),
  (LR(l='노래', l_tag='Noun', r='', r_tag=None, b=13, m=15, e=15), 0.5),
  (LR(l='노래', l_tag='Noun', r='입니다', r_tag='Josa', b=13, m=15, e=18), 0.95)]]


만약 특정 품사의 단어에 대하여 점수의 가중치를 더하고 싶다면 preference 를 이용할 수 있습니다. 

dict[tag][word] = score 형식의 dict-dict 인 preference 를 Evaluator 에 넣어주면 됩니다. debug mode 로 확인하면 해당 단어의 점수가 더해졌음을 볼 수 있습니다. 

In [27]:
preference = {
    'Noun': {'아이오아이':10.0, '너무너무너무':5}
}

evaluator = LREvaluator(preference=preference)
tagger = SimpleTagger(generator, evaluator, postprocessor)
tags, debugs = tagger.tag(sent, debug=True)

pprint(debugs)

[[(LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=0, m=2, e=2), 0.4),
  (LR(l='너무너무너무', l_tag='Noun', r='', r_tag=None, b=0, m=6, e=6), 5.9),
  (LR(l='너무너무너무', l_tag='Noun', r='는', r_tag='Josa', b=0, m=6, e=7), 6.15),
  (LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=2, m=4, e=4), 0.4),
  (LR(l='너무', l_tag='Adverb', r='', r_tag=None, b=4, m=6, e=6), 0.4),
  (LR(l='아이', l_tag='Noun', r='', r_tag=None, b=7, m=9, e=9), 0.5),
  (LR(l='아이오아이', l_tag='Noun', r='', r_tag=None, b=7, m=12, e=12), 10.8),
  (LR(l='아이오아이', l_tag='Noun', r='의', r_tag='Josa', b=7, m=12, e=13), 11.05),
  (LR(l='아이', l_tag='Noun', r='', r_tag=None, b=10, m=12, e=12), 0.5),
  (LR(l='아이', l_tag='Noun', r='의', r_tag='Josa', b=10, m=12, e=13),
   0.7499999999999999),
  (LR(l='노래', l_tag='Noun', r='', r_tag=None, b=13, m=15, e=15), 0.5),
  (LR(l='노래', l_tag='Noun', r='입니다', r_tag='Josa', b=13, m=15, e=18), 0.95)]]
